In [1]:
pip install torch transformers datasets sentencepiece

In [2]:
import torch
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset, load_metric

In [3]:
import pandas as pd
from datasets import Dataset
# Load the CSV file into a pandas DataFrame
df = pd.read_csv('/content/summarizdataset.csv', encoding='utf-8')
df

,text,type,Processed Text,summarizer
0,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...,culture,اشرف رئيس الجمهوريه الباجي قايد السبسي اليوم ب...,\nأشرف رئيس الجمهورية الباجي قايد السبسي اليوم...
1,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون...",culture,تحصل كتاب المصحف وقراءاته الفه باحثون تونسيون ...,"\nتحصل كتاب ""المصحف وقراءاته"" الذي ألفه باحثون..."
2,\nاحتضن جناح تونس في القرية الدولية للأفلام بم...,culture,احتضن جناح تونس القريه الدوليه للافلام بمدينه ...,تونس حاضرة من جهة أخرى ستكون تونس حاضرة في قائ...
3,\nشهدت برلين أمس الجمعة افتتاح مسجد فريد من نو...,culture,شهدت برلين الجمعه افتتاح مسجد فريد نوعه الاقل ...,واستأجرت صاحبة المشروع المحامية والكاتبة سيران...
4,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...,culture,نعت وزاره المنشد عز بن محمود انتقل جوار يوم تن...,\nنعت وزارة الشّؤون الثّقافيّة المنشد الصّوفي ...
...,...,...,...,...
8373,\nتأجل الإضراب العام في قطاع الصحة الذي كان مق...,localnews,تاجل الاضراب العام قطاع الصحه مقررا تنفيذه الي...,وكانت انعقدت عشية أمس في وزارة الشؤون الاجتماع...
8374,\nكشف الناشطان كريم نوار وعفيف زقية اللذان أشر...,localnews,كشف الناشطان كريم نوار وعفيف زقيه اشرفا عمليه ...,وأكدا ان تكلفة العلم لم تتجاوز 3 آلاف دينار مق...
8375,\nتمكّنت فرقة الأبحاث والتفتيش للحرس الوطني بط...,localnews,فرقه الابحاث والتفتيش للحرس الوطني بطبلبه ولاي...,كما تمّ إلقاء القبض على عنصر رابع (عمره 32 سنة...
8376,\nقرر الأهالي بمناطق هيشر وعين القارصي والغولا...,localnews,قرر الاهالي بمناطق هيشر وعين القارصي والغولايث...,وتأتي هذه الخطوة على خلفية ما اعتبره أهالي هذه...


In [4]:
df = df.sample(frac=0.2, random_state=42)
df

,text,type,Processed Text,summarizer
4730,\nتمكن أعوان الإدارة المركزية لمكافحة الإرهاب ...,localnews,تمكن اعوان الاداره المركزيه لمكافحه الارهاب با...,وبعد التنسيق مع النيابة العمومية، أذنت بالإحتف...
8168,\nيكون الطقس مغيّما جزئيّا بأغلب الجهات.\nالري...,localnews,يكون الطقس باغلب الجهات الريح تكون القطاع الشر...,البحر يكون قليل الاضطراب فمضطربا بالشمال وتترا...
2333,\nكشف السياسي نجيب الشابي اليوم الثلاثاء عن خط...,politic,كشف السياسي نجيب الشابي اليوم الثلاثاء خطته لا...,\nكشف السياسي نجيب الشابي اليوم الثلاثاء عن خط...
5426,\nعبّر عدد من أهالي مدينة القلعة الكبرى عن است...,localnews,عدد اهالي مدينه القلعه الكبرى استيائهم الانقطا...,\nعبّر عدد من أهالي مدينة القلعة الكبرى عن است...
5407,\nأثارت صورة من مجلس مخصص لمناقشة قضايا نسائية...,localnews,اثارت صوره مجلس مخصص لمناقشه قضايا نسائيه يحمل...,وكان الأمير فيصل بن مشعل بن سعود، حاكم منطقة ا...
...,...,...,...,...
2858,\nاصدرت اللجنة الوطنية الاولمبية التونسية بلاغ...,sport,اصدرت اللجنه الوطنيه الاولمبيه التونسيه بلاغا ...,\nاصدرت اللجنة الوطنية الاولمبية التونسية بلاغ...
1129,\nقالت مسؤولة السياسة الخارجية في الاتحاد الأو...,internationalNews,قالت مسؤوله السياسه الخارجيه الاتحاد الاوروبي ...,"وأكدت موغيريني أن ""الهجرة لا يمكن أن تدار إلا ..."
3980,"\nنفت مصادر طبية مطلعة ل""الجوهرة أف أم"" خبر وف...",society,نفت مصادر طبيه مطلعه الجوهره خبر وفاه الطفله ت...,"\nنفت مصادر طبية مطلعة ل""الجوهرة أف أم"" خبر وف..."
7563,"\nنشرت صفحة ""هنا قرطاج"" التابعة لرئاسة الجمهور...",localnews,نشرت صفحه قرطاج التابعه لرئاسه الجمهوريه اليوم...,"وخاطب الليبيين قائلا إن ""بلادهم ستصبح كتونس، و..."


In [5]:
# Convert the DataFrame to a Dataset object
dataset = Dataset.from_pandas(df)

In [6]:
dataset

Dataset({
    features: ['text', 'type', 'Processed Text', 'summarizer', '__index_level_0__'],
    num_rows: 1676
})

In [7]:
model_name = "facebook/mbart-large-cc25"

tokenizer = MBartTokenizer.from_pretrained(model_name, src_lang="ar_AR", tgt_lang="ar_AR")

def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['summarizer']

    # Tokenize inputs with padding and truncation
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding='max_length')

    # Tokenize targets with padding and truncation
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    # Add padding to labels to match the model's input shape
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Apply preprocessing to the entire dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'summarizer'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/1676 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
print(tokenized_datasets.column_names)

['type', 'Processed Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels']


In [9]:
import pandas as pd
from datasets import Dataset

# Convert tokenized dataset to DataFrame
df = pd.DataFrame(tokenized_datasets)

In [10]:
df

,type,Processed Text,__index_level_0__,input_ids,attention_mask,labels
0,localnews,تمكن اعوان الاداره المركزيه لمكافحه الارهاب با...,4730,"[128195, 61731, 12880, 94996, 208496, 795, 954...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[48881, 878, 159230, 1546, 212634, 216636, 50,..."
1,localnews,يكون الطقس باغلب الجهات الريح تكون القطاع الشر...,8168,"[11194, 35717, 94129, 68513, 179, 1168, 2889, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[73548, 11194, 174969, 3484, 156698, 5042, 210..."
2,politic,كشف السياسي نجيب الشابي اليوم الثلاثاء خطته لا...,2333,"[118574, 63776, 1706, 106912, 161309, 179, 708...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[118574, 63776, 1706, 106912, 161309, 179, 708..."
3,localnews,عدد اهالي مدينه القلعه الكبرى استيائهم الانقطا...,5426,"[6, 27949, 1168, 835, 12581, 230, 1333, 176, 2...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[6, 27949, 1168, 835, 12581, 230, 1333, 176, 2..."
4,localnews,اثارت صوره مجلس مخصص لمناقشه قضايا نسائيه يحمل...,5407,"[217278, 368, 71996, 230, 9580, 665, 107780, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[28187, 111407, 147532, 2567, 11924, 49037, 25..."
...,...,...,...,...,...,...
1671,sport,اصدرت اللجنه الوطنيه الاولمبيه التونسيه بلاغا ...,2858,"[878, 48811, 368, 84341, 50910, 76804, 376, 35...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[878, 48811, 368, 84341, 50910, 76804, 376, 35..."
1672,internationalNews,قالت مسؤوله السياسه الخارجيه الاتحاد الاوروبي ...,1129,"[90621, 55468, 250, 159514, 55317, 240, 44475,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[103469, 368, 5086, 85517, 44180, 862, 44, 422..."
1673,society,نفت مصادر طبيه مطلعه الجوهره خبر وفاه الطفله ت...,3980,"[27498, 87729, 81901, 250, 132158, 250, 795, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[27498, 87729, 81901, 250, 132158, 250, 795, 5..."
1674,localnews,نشرت صفحه قرطاج التابعه لرئاسه الجمهوريه اليوم...,7563,"[186148, 142414, 44, 176, 1069, 24637, 50392, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[65, 229428, 173118, 755, 134050, 5202, 44, 78..."


In [12]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame(tokenized_datasets.to_pandas()) # Keep original column names

# Split the DataFrame into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert DataFrames back to Datasets
train_dataset = Dataset.from_pandas(train_df) # Now column names should match
val_dataset = Dataset.from_pandas(val_df)


CastError: Couldn't cast
type: string
Processed Text: string
__index_level_0__: int64
input_ids: list<item: int32>
  child 0, item: int32
attention_mask: list<item: int8>
  child 0, item: int8
labels: list<item: int64>
  child 0, item: int64
__index_level_0__: int64
-- schema metadata --
pandas: '{"index_columns": ["__index_level_0__"], "column_indexes": [{"na' + 1065
to
{'type': Value(dtype='string', id=None), 'Processed Text': Value(dtype='string', id=None), '__index_level_0__': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}
because column names don't match

In [13]:
from datasets import DatasetDict

# Create a DatasetDict with 'train' and 'validation' splits
# Split the original dataset into train and validation sets (adjust split ratio as needed)
train_dataset = tokenized_datasets.train_test_split(test_size=0.1, seed=42)['train']
val_dataset = tokenized_datasets.train_test_split(test_size=0.1, seed=42)['test']

tokenized_datasets = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset
})

In [14]:
from datasets import DatasetDict

# Assuming tokenized_datasets is a DatasetDict with 'train' and 'validation'
tokenized_datasets = DatasetDict({
    'train': tokenized_datasets['train'],
    'validation': tokenized_datasets['validation']
})


In [15]:
model = MBartForConditionalGeneration.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

In [26]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Reduced batch size
    per_device_eval_batch_size=1,  # Reduced batch size
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),  # Enable fp16 for memory efficiency
    logging_dir='./logs',
    logging_steps=10,
    save_steps=1000,
    gradient_accumulation_steps=4,  # Added gradient accumulation
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
!pip install rouge_score

In [28]:
from datasets import load_metric

metric = load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: v.mid.fmeasure * 100 for k, v in result.items()}


In [29]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [30]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.111500,0.063237,7.746490,2.980700,7.523166,7.722996
2,0.111900,0.025142,27.741101,13.789683,27.306397,27.765903


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.111500,0.063237,7.746490,2.980700,7.523166,7.722996
2,0.111900,0.025142,27.741101,13.789683,27.306397,27.765903
3,0.048700,0.021827,28.984552,15.666808,27.734788,28.952192


TrainOutput(global_step=1131, training_loss=0.44663720125057443, metrics={'train_runtime': 5969.2534, 'train_samples_per_second': 0.758, 'train_steps_per_second': 0.189, 'total_flos': 9804095070142464.0, 'train_loss': 0.44663720125057443, 'epoch': 3.0})

In [31]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.021827416494488716, 'eval_rouge1': 28.984552154194997, 'eval_rouge2': 15.666808390022677, 'eval_rougeL': 27.734788359788354, 'eval_rougeLsum': 28.952191987906268, 'eval_runtime': 614.1782, 'eval_samples_per_second': 0.274, 'eval_steps_per_second': 0.274, 'epoch': 3.0}


In [32]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/sentencepiece.bpe.model',
 './saved_model/added_tokens.json')

In [37]:
!zip -r /content/sample_data.zip /content/sample_data

  adding: content/sample_data/ (stored 0%)
  adding: content/sample_data/anscombe.json (deflated 83%)
  adding: content/sample_data/README.md (deflated 42%)
  adding: content/sample_data/california_housing_test.csv (deflated 76%)
  adding: content/sample_data/mnist_test.csv (deflated 88%)
  adding: content/sample_data/california_housing_train.csv (deflated 79%)
  adding: content/sample_data/mnist_train_small.csv (deflated 88%)


In [38]:
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/config.json (deflated 60%)
  adding: content/saved_model/generation_config.json (deflated 35%)
  adding: content/saved_model/special_tokens_map.json (deflated 57%)
  adding: content/saved_model/tokenizer_config.json (deflated 90%)
  adding: content/saved_model/sentencepiece.bpe.model (deflated 49%)
  adding: content/saved_model/model.safetensors (deflated 7%)


In [39]:
!zip -r /content/saved_model.zip /content/saved_model
from google.colab import files
files.download("/content/saved_model.zip")


updating: content/saved_model/ (stored 0%)
updating: content/saved_model/config.json (deflated 60%)
updating: content/saved_model/generation_config.json (deflated 35%)
updating: content/saved_model/special_tokens_map.json (deflated 57%)
updating: content/saved_model/tokenizer_config.json (deflated 90%)
updating: content/saved_model/sentencepiece.bpe.model (deflated 49%)
updating: content/saved_model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
trainer.save_model("./my_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'num_beams': 5, 'forced_eos_token_id': 2}


In [42]:
!zip -r /content/my_model.zip /content/my_model
from google.colab import files
files.download("/content/my_model.zip")

  adding: content/my_model/ (stored 0%)
  adding: content/my_model/config.json (deflated 60%)
  adding: content/my_model/generation_config.json (deflated 35%)
  adding: content/my_model/special_tokens_map.json (deflated 57%)
  adding: content/my_model/tokenizer_config.json (deflated 90%)
  adding: content/my_model/sentencepiece.bpe.model (deflated 49%)
  adding: content/my_model/training_args.bin (deflated 52%)
  adding: content/my_model/model.safetensors (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
from transformers import MBartForConditionalGeneration, MBartTokenizer
import torch

# Path to the saved model directory
model_path = "./my_model"

# Load the tokenizer and model
tokenizer = MBartTokenizer.from_pretrained(model_path)
model = MBartForConditionalGeneration.from_pretrained(model_path)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): MBartScaledWordEmbedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (fi

In [45]:
def prepare_input(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True, padding='max_length')

    # Move inputs to the device if using GPU
    inputs = {key: val.to(device) for key, val in inputs.items()}
    return inputs


In [46]:
def generate_summary(text):
    # Prepare the input
    inputs = prepare_input(text)

    # Generate summary
    with torch.no_grad():
        summary_ids = model.generate(**inputs, max_length=150, num_beams=5, length_penalty=2.0, early_stopping=True)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [48]:
# Example text to summarize
text = 'نظم اتحاد كتاب وأدباء الإمارات- فرع أبوظبي، مساء أمس الأول، في مقره بالمسرح الوطني أمسية احتفائية بالشاعر العراقي بدر شاكر السيّاب بمناسبة مرور 50 عاماً على رحيله، وتضمنت قراءات شعرية أدائية لأعمال السيّاب، حضرها عدد من الكتاب والأدباء . استهل الأمسية الشاعر والتشكيلي محمد المزروعي، رئيس الهيئة الإدارية في الاتحاد فرع أبوظبي، بتسليط الضوء على أهمية السيّاب، كعلم من أعلام الأدب العربي، حيث أقر المؤتمر العام الخامس والعشرون للاتحاد العام للأدباء والكُتاب العرب، في المنامة ديسمبر/ كانون الأول الماضي، أن يخصص عام 2014 للسياب، من خلال فعاليات ثقافية وأدبية بهذه المناسبة في كل الاتحادات الأعضاء، وبالتعاون مع الأمانة العامة، كما اقترح المؤتمر تعميم هذا التقليد بحيث يصبح كل عام مخصصاً لعلم من أعلام الأدب العربي .الشاعر والتشكيلي العراقي حماد الهاشمي ارتحل في عوالم السيّاب الفسيحة، وجال في فضاءاته الرحبة مستحضراً ملامح تجربته الشعرية، وقال: إن النفحة الروحية التي ألمسها في شعر السيّاب هي التي دفعتني إلى الغوص في أعماق انتاجه الشعري الزاخر .وأشار الهاشمي إلى أن تجربة السيّاب الشعرية تمتاز بالعديد من الخصائص الفارقة من بينها لغته الجميلة الطافحة، وصوره البديعة، وأسلوبه السلس مما دفعه للتماهي مع روح السيّاب الشاعرة، واستكشاف عالمها الآسر . قرأ الهاشمي جيكور تجالسنا في المنفى وقصائد أخرى تحتفي بتجربة للسياب الشعرية تبحر في أمواج بحره المتلاطمة .الشاعر المصري إبراهيم عبد الفتاح تساءل في البداية ألم يحن الوقت أن نكتب عن السيّاب؟ ثم قرأ نصه الذي يقول فيه: خمسون عاماً من الخرس كانت كافية ليقايض دجلة ماءه بالخل/ قال الفتى لحبيبته: تكفيني قصيدة واحدة/ وطرف ثوبك وصوتي/ فقنابل الشرطي خائفة/ ولن تسيل الوطن/ لكنها سبقته لبغداد واعتصمت/ حتى يضع قلبه مكان اللون بالعلم .'


# Generate summary
summary = generate_summary(text)

# Print the summary
print("Summary:", summary)


Summary: قرأ الهاشمي إلى أن تجربة السيّاب الشعرية تمتاز بالعديد من الخصائص الفارقة من بينها لغته الجميلة الطافحة، وصوره البديعة، وأسلوبه السلس مما دفعه للتماهي مع روح السيّاب الشاعرة، واستكشاف عالمها الآسر.
